In [1]:
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Lambda, Bidirectional, LSTM, Dense
from keras_bert import load_trained_model_from_checkpoint
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps
from datetime import datetime
from collections import Counter
import keras.callbacks
import re
import codecs
import time

Using TensorFlow backend.


In [2]:
import os
import codecs
import pandas as pd
import numpy as np

bert_dir = r'C:\Users\rocker\Python - deep learning\Deep learning\keras 大神\bert'
config_path = os.path.join(bert_dir, 'bert_config.json')
checkpoint_path = os.path.join(bert_dir, 'bert_model.ckpt')
dict_path = os.path.join(bert_dir, 'vocab.txt')

# 0. 載入資料

In [3]:
data = pd.read_csv(r'C:\Users\rocker\Python - deep learning\Deep learning\keras 大神\content_df_0620.csv')
data = data[data["status"]=="ok"].drop(["url","context","raw_content","status", "content_status"],axis = 1)
data['aml_label'] = data['name'].apply(lambda x: 0 if x == '[]' else 1)
data['name'] = data['name'].apply(lambda x: eval(x))

In [4]:
data['content'] = data['content'].apply(lambda x: re.sub('<[^>]*>|【[^】]*】|（[^）]*）|〔[^〕]*〕', '', x))
data['content'] = data['content'].apply(lambda x: x.strip().replace('記者', '＜')
                                                   .replace('報導', '＞')
                                                   .replace('▲', '')\
                                                   .replace('。　', '。')\
                                                   .replace('', '')\
                                                   .replace('.', '')\
                                                   .replace(' ', '')\
                                                   .replace('“', '「')\
                                                   .replace('”', '」'))
data['content'] = data['content'].apply(lambda x: re.sub('＜[^＞]*＞', '', x))

In [5]:
train = data

In [6]:
data2 = pd.read_csv(r'C:\Users\rocker\extra data\人肉爬蟲_20200615.csv', encoding='cp950')
data2['name'] = data2['name'].apply(lambda x: eval(x))
data2['content'] = data2['content'].apply(lambda x: re.sub('<[^>]*>|【[^】]*】|（[^）]*）|〔[^〕]*〕', '', x))
data2['content'] = data2['content'].apply(lambda x: x.strip().replace('記者', '＜')
                                                   .replace('報導', '＞')
                                                   .replace('▲', '')\
                                                   .replace('。　', '。')\
                                                   .replace('', '')\
                                                   .replace('.', '')\
                                                   .replace(' ', '')\
                                                   .replace('“', '「')\
                                                   .replace('”', '」'))
data2['content'] = data2['content'].apply(lambda x: re.sub('＜[^＞]*＞', '', x))
test = data2
test = test[['news_id', 'name', 'content', 'aml_label']]

In [7]:
#train, test =  train_test_split(data, test_size = 0.2, random_state = 0)

# 1. 建立 aml 模型

In [8]:
def create_tokenizer(dict_path):
    
    token_dict = {}
    with codecs.open(dict_path, 'r', 'utf8') as reader:
        for line in reader:
            token = line.strip()
            token_dict[token] = len(token_dict)
            
    return token_dict

def transfer(i):
    
    if i != 0:
        return 1
    else:
        return 0

def encoded(tokenizer, data, maxlen):
    
    x, y, z = [], [], []
    if 'content' in data.columns:
        for content in data['content']:
            x1, x2 = tokenizer.encode(content, max_len=maxlen)
            x3 = [transfer(i) for i in x1]
            x.append(x1)
            y.append(x2)
            z.append(x3)
    elif 'Sentence' in data.columns:
        for content in data['Sentence']:
            x1, x2 = tokenizer.encode(content, max_len=maxlen)
            x3 = [transfer(i) for i in x1]
            x.append(x1)
            y.append(x2)
            z.append(x3)
            
    return x, y, z

In [9]:
token_dict = create_tokenizer(dict_path)
tokenizer = Tokenizer(token_dict)

In [10]:
maxlen = 256
batch_size = 8
epochs = 3

In [11]:
def bert_LSTM_model():
    
    model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, seq_len=maxlen)
    sequence_output = model.layers[-9].output
    #sequence_output = Lambda(lambda x: x[:, 0])(sequence_output)
    sequence_output = Bidirectional(LSTM(128, return_sequences=False))(sequence_output)
    output = Dense(1, activation='sigmoid')(sequence_output)
    model = Model(model.input, output)
    
    for layer in model.layers:
        layer.trainable = False
    model.layers[-1].trainable = True
    model.layers[-2].trainable = True
    return model

model = bert_LSTM_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
#def bert_model():
#    
#    model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, seq_len=maxlen)
#    sequence_output = model.layers[-6].output
#    sequence_output = Dense(128, activation='relu')(sequence_output)
#    output = Dense(1, activation='sigmoid')(sequence_output)
#    model = Model(model.input, output)
#    return model
#
#model = bert_model()
#
#set_trainable = False
#for layer in model.layers:
#    if layer.name == 'Encoder-12-MultiHeadSelfAttention':
#        set_trainable = True
#    if set_trainable:
#        layer.trainable = True
#    else:
#        layer.trainable = False

In [ ]:
label = np.asarray(train['aml_label'])

In [ ]:
input_id, segment_id, mask_input = encoded(tokenizer, train, maxlen)

In [ ]:
total_steps, warmup_steps = calc_train_steps(
    num_example=train.shape[0],
    batch_size=batch_size,
    epochs=epochs,
    warmup_proportion=0.1,
)

optimizer = AdamWarmup(total_steps, warmup_steps, lr=1e-3, min_lr=1e-5)

In [ ]:
callback_list = [
                 keras.callbacks.EarlyStopping(monitor='val_acc', patience=1)
                 #,ModelCheckpoint(filepath='AML_bert.h5', monitor='val_loss', save_best_only=True)
                ]

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

model.fit([input_id, segment_id, mask_input],
          label,
          epochs=epochs,
          batch_size=batch_size,
          #validation_split=0.1,
          #callbacks=callback_list
         )

In [12]:
# 載入模型
model.load_weights(r'C:\Users\rocker\bert model\model1.h5')

# 2. 建立 NER 模型
### transfer_NER 要跑 1.5 小時，訓練模型要跑 17mins/epoch，所以load weight就好

In [13]:
# 把大於512的新聞以句點分段 (bert最多只能吃512)
def split_content(data):
    data_more_split = pd.DataFrame()
    for i, row in data.iterrows():
        if (len(row['content']) > 512) & (len(row['content']) <= 1024):

            s = row['content']
            s_split = [(i, abs(len(s)//2 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left = min(s_split, key=lambda x: x[1])[0]
            first = "。".join([s_split[i][2] for i in range(idx_left)])
            second = "。".join([s_split[i][2] for i in range(idx_left, len(s_split))])    
            contents = [first, second]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content}, index=[66]), ignore_index=True)

        elif len(row['content']) > 1024:

            s = row['content']
            s_split1 = [(i, abs(len(s)//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            s_split2 = [(i, abs(len(s)*2//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left1 = min(s_split1, key=lambda x: x[1])[0]
            idx_left2 = min(s_split2, key=lambda x: x[1])[0]
            first = "。".join([s_split1[i][2] for i in range(idx_left1)])
            second = "。".join([s_split1[i][2] for i in range(idx_left1, idx_left2)])
            third = "。".join([s_split1[i][2] for i in range(idx_left2, len(s_split1))])
            contents = [first, second, third]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content}, index=[66]), ignore_index=True)
    
    return data_more_split

In [14]:
maxlen_ner = 512
batch_size = 8
epochs = 3
input_shape = (maxlen_ner, )

In [15]:
def bert_BiLSTM_CRF_model():
    
    ner_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, seq_len=maxlen_ner)
    bert_output = ner_model.layers[-9].output
    X = Lambda(lambda x: x[:, 0: input_shape[0]])(bert_output)
    X = Bidirectional(LSTM(128, return_sequences=True))(X)
    #X = TimeDistributed(Dense(len(y_token_dict), activation='relu'))(X)
    output = CRF(3, sparse_target = True)(X)    
    ner_model = Model(ner_model.input, output)
    
    for layer in ner_model.layers:
        layer.trainable = False
    ner_model.layers[-1].trainable = True
    ner_model.layers[-2].trainable = True
    
    return ner_model

In [16]:
ner_model = bert_BiLSTM_CRF_model()

In [17]:
# transfer_NER 要跑 1.5 小時，訓練模型要跑 17mins/epoch，所以load weight就好
ner_model.load_weights(r'C:\Users\rocker\bert model\ner_model.h5')

# 3. 建立模型 2

In [ ]:
train = train[train['aml_label'] == 1]

In [ ]:
data2 = pd.read_csv(r'E:\Desktop\檢察官_法官_filter3 - 複製.csv', encoding='cp950')
data2['aml_label'] = data2['people_list'].apply(lambda x: 0 if x == '[]' else 1)
data2 = data2[data2['aml_label']==1]
data2 = data2.drop(['title', 'link'], axis=1)
data2['people_list'] = data2['people_list'].apply(lambda x: eval(x))
data2['news_id'] = range(20000, 20000+len(data2))
data2['content'] = data2['content'].apply(lambda x: re.sub('<[^>]*>|【[^】]*】|（[^）]*）|〔[^〕]*〕', '', x))
data2['content'] = data2['content'].apply(lambda x: x.strip().replace('記者', '＜')
                                                   .replace('報導', '＞')
                                                   .replace('▲', '')\
                                                   .replace('。　', '。')\
                                                   .replace('', '')\
                                                   .replace('.', '')\
                                                   .replace(' ', '')\
                                                   .replace('“', '「')\
                                                   .replace('”', '」'))
data2['content'] = data2['content'].apply(lambda x: re.sub('＜[^＞]*＞', '', x))
data2.columns = ['news_id', 'content', 'aml_label', 'name']

In [ ]:
data3 = pd.read_csv(r'E:\Desktop\檢察官_法官_filter.csv', encoding='cp950')
data3 = data3.drop(['title', 'link'], axis=1)
data3['aml_label'] = data3['people_list'].apply(lambda x: 0 if x == '[]' else 1)
data3 = data3[data3['aml_label']==1]
data3['people_list'] = data3['people_list'].apply(lambda x: eval(x))
data3['news_id'] = range(30000, 30000+len(data3))
data3.columns = ['news_id', 'content', 'aml_label', 'name']

In [ ]:
data4 = pd.read_csv(r'E:\Desktop\nownews_all_content_part1_0708.csv', encoding='cp950')
data4 = data4.drop(['title', 'link', 'people_list'], axis=1)
data4['aml_label'] = data4['name'].apply(lambda x: 0 if x == '[]' else 1)
data4 = data4[data4['aml_label']==1]
data4['name'] = data4['name'].apply(lambda x: eval(x))
data4['news_id'] = range(40000, 40000+len(data4))
data4['content'] = data4['content'].apply(lambda x: re.sub('<[^>]*>|【[^】]*】|（[^）]*）|〔[^〕]*〕', '', x))
data4['content'] = data4['content'].apply(lambda x: x.strip().replace('記者', '＜')
                                                   .replace('報導', '＞')
                                                   .replace('▲', '')\
                                                   .replace('。　', '。')\
                                                   .replace('', '')\
                                                   .replace('.', '')\
                                                   .replace(' ', '')\
                                                   .replace('“', '「')\
                                                   .replace('”', '」'))
data4['content'] = data4['content'].apply(lambda x: re.sub('＜[^＞]*＞', '', x))

In [ ]:
train = train.append(data2).append(data3).append(data4)

In [18]:
maxlen_sentences = 256
batch_size = 8
epochs = 3

In [19]:
# 把大於512的新聞以句點分段 (bert最多只能吃512)
def split_content_model2(data):
    data_more_split = pd.DataFrame()
    for i, row in data.iterrows():
        if (len(row['content']) > 512) & (len(row['content']) <= 1024):

            s = row['content']
            s_split = [(i, abs(len(s)//2 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left = min(s_split, key=lambda x: x[1])[0]
            first = "。".join([s_split[i][2] for i in range(idx_left)])
            second = "。".join([s_split[i][2] for i in range(idx_left, len(s_split))])    
            contents = [first, second]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content, 'aml_label':row['aml_label']}, index=[66]), ignore_index=True)

        elif len(row['content']) > 1024:

            s = row['content']
            s_split1 = [(i, abs(len(s)//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            s_split2 = [(i, abs(len(s)*2//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left1 = min(s_split1, key=lambda x: x[1])[0]
            idx_left2 = min(s_split2, key=lambda x: x[1])[0]
            first = "。".join([s_split1[i][2] for i in range(idx_left1)])
            second = "。".join([s_split1[i][2] for i in range(idx_left1, idx_left2)])
            third = "。".join([s_split1[i][2] for i in range(idx_left2, len(s_split1))])
            contents = [first, second, third]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content, 'aml_label':row['aml_label']}, index=[66]), ignore_index=True)
    
    return data_more_split

In [20]:
def split2textByName(news, name):

    def _addperiod(matched):
        intStr = matched.group()
        addedStr = str('。='+intStr)
        return addedStr
    
    i = np.sum([[news.count(name) for news in news] for name in name], axis=0)
    
    if type(i) != np.ndarray:
        return news
    else:
        for i,(num, text) in zip(list(i), enumerate(news)):
            if (i > 1) & ('、' not in text) & (('之' in text) | ('的' in text)):
                replacedStr = re.sub('|'.join(name), _addperiod, text)
                replacedStr = re.sub('。=','', replacedStr, 1)
                news[num] = replacedStr
        newsAfter = sum([news.split('=') for news in news], [])

        return newsAfter

In [21]:
def predict_sentences_model2(test, people_list, stick, tokenizer=tokenizer, maxlen=maxlen_sentences):
    
    AML = pd.DataFrame(columns=['news_id', 'Name', 'Sentence'])
    aml_highrisk = np.asarray(test['content'])
    news_ids = np.asarray(test['news_id'])
    people_list = list(test['people_list'])

    
    for k, (news_id, y_news) in enumerate(zip(news_ids ,aml_highrisk)): 
        #用，。？切分句子
        y_news = y_news.replace('。','=。')
        y_news = y_news.replace('，','*，')
        y_news = y_news.replace('？','+？')
        y_news = y_news.replace('；','{；')
        
        news = re.split('，|。|？|；', y_news)
        
        news = [news.replace('=','。') for news in news]
        news = [news.replace('*','，') for news in news]
        news = [news.replace('+','？') for news in news]
        news = [news.replace('{','；') for news in news]
        news = split2textByName(news, people_list[k])
#############################################################################################################
        for i in range(len(people_list[k])):
            # 找出人名存在的 news index
            index = [index for index, _ in enumerate(news) if people_list[k][i] in _]                                
            x = [people for people in people_list[k] if people_list[k][i] not in people]

            name = [name for name in people_list[k] if name != people_list[k][i]]
            name = [name for name in name if ((len(name)<3) & (name[0] != people_list[k][i][0])) | (len(name)>=3)]
            name.sort(reverse=True)

            #if name != []:
            #    text = [re.sub('|'.join(name), '其他人名', news) for news in news]
            #else:
            #    text = news

            for j in index:

                mid = news[j]

                if name != []:
                    name = [name.replace('?', '\?') for name in name]
                    mid = re.sub('|'.join(name), '', mid)
                
                if j == 0:
                    if len(news) != 1:
                        end = news[j+1]  

                        if True in [people in news[j+1] for people in x]:
                            sentences = mid
                        else:
                            sentences = mid + end
                    elif len(news) == 1:
                        sentences = mid

                elif j+1 == len(news):
                    start = news[j-1]

                    if True in [people in news[j-1] for people in x]:
                        sentences = mid
                    else:
                        sentences = start + mid

                else:
                    end = news[j+1]
                    start = news[j-1]                    

                    if '。' in start:
                        start = ''
                    elif '。' in mid:
                        end = ''                 

                    if (True not in [people in news[j-1] for people in x]) & (True not in [people in news[j+1] for people in x]):
                        sentences = start + mid + end
                    elif (True not in [people in news[j-1] for people in x]) & (True in [people in news[j+1] for people in x]):
                        sentences = start + mid
                    elif (True in [people in news[j-1] for people in x]) & (True not in [people in news[j+1] for people in x]):
                        sentences = mid + end
                    else:
                        sentences = mid    
                        
                AML = AML.append(pd.DataFrame([[news_ids[k] ,people_list[k][i], sentences]], columns=AML.columns))
                #break  
                
    #若 stick==True 則把多筆同姓名句子以逗點合併 (效果不好)
    if stick:
        AML = AML.groupby(['news_id', 'Name'])['Sentence'].apply('，'.join).reset_index()
    
    
    # 把指向同一人的姓名改成一樣（陳男 -> 陳水扁），若指向多人則不改（陳男 -> 陳致中、陳水扁）
    # 將預測不完整的名字回填（王音 -> 王音之）
    name_list = []
    for ids in AML['news_id'].unique():
        full_name = [name for name in AML[(AML['news_id'] == ids)]['Name']]
        full_3name = [name for name in AML[(AML['news_id'] == ids)]['Name'] if len(name) == 3]
        
        a = Counter([name[0] for name in full_3name])
        keep = [k for k,v in a.items() if v == 1]
        full_3name_filter = [name for name in full_3name if name[0] in keep]
        name_dict = dict((name[0], name) for name in full_3name_filter)   # ex: {'陳' : '陳水扁'}
        name_dict_2 = dict(zip([name[0:2] for name in full_3name], full_3name))  # ex: {'王音': '王音之'}
        name_dict_3 = dict(zip([name[1:] for name in full_3name], full_3name))
        for name in full_name:
            if (name[0] in name_dict.keys()) & (len(name) == 1):
                name_list.append(name_dict.get(name[0]))
            elif (name[0] in name_dict.keys()) & (len(name) == 2) & (name[-1] in ['男', '嫌', '婦', '夫', '某', '女', '妻',\
                                                                                  '員', '稱', '家', '哥', '媽', '生', '處',\
                                                                                  '和', '揆', '要', '再', '董', '涉', '母',\
                                                                                  '辱', '公', '少', '為', '指', '翁', '粉',\
                                                                                  '趁', '仔', '依', '氏', '父']):
                name_list.append(name_dict.get(name[0]))
            elif (name in name_dict_2.keys()) & (len(name) == 2):
                name_list.append(name_dict_2.get(name))
            elif (name in name_dict_3.keys()):
                name_list.append(name_dict_3.get(name))
            else:
                name_list.append(name)
                
    
    # 排除重複資料、排除一字、兩字簡稱、兩字三字四字姓不在姓氏表中的人
    AML['Name'] = name_list
    AML = AML.drop_duplicates()
    AML = AML[AML['Name'].apply(lambda x: (len(x) > 1) )]
    AML = AML[~AML['Name'].apply(lambda x: (len(x) == 2) & (x[1] in ['男', '嫌', '婦', '夫', '某', '女', '妻',\
                                                                     '員', '稱', '家', '哥', '媽', '生', '處',\
                                                                     '和', '揆', '要', '再', '董', '涉', '母',\
                                                                     '辱', '公', '少', '為', '指', '翁', '粉',\
                                                                     '趁', '仔', '依', '氏', '父']))]
    
    return AML

In [22]:
# 取得名字 (預測結果為onehot的狀態)
def get_name(input_id, y_pred):
    
    label_list = []
    word_dict = {v: k for k, v in token_dict.items()}
    
    for input_data, y in zip(input_id, y_pred):
        people_index = ''.join([str(a) for a in list(y)])
        j = 0
        name_list = []
        split_index = re.findall('[12]2*', people_index)
        name = ''.join([word_dict.get(input_data[index]) for index, value in enumerate(y) if value != 0])
        
        # [UNK], [PAD]會被算成 5 個字元，避免轉換成文字的index因長度不同對不上，故用 1 個字元的其他符號替代
        # 王春甡 -> 王春[UNK] -> 王春?
        name = name.replace('[UNK]','?')
        name = name.replace('[PAD]','!')
        
        for i in split_index:
            name_list.append(name[0+j:len(i)+j])
            j = len(i) + j
            
        name_list = [name for name in name_list]
        label_list.append(list(set(name_list)))
    
    return label_list

In [ ]:
# 2. 將超過 512 的句子以句點拆成多句分段預測
train_ner = train.drop(['name'], axis=1)
data_less = train_ner[train_ner['content'].str.len() <= 512]
data_more = train_ner[train_ner['content'].str.len() > 512]
data_more_split = split_content_model2(data_more)
train_ner = data_less.append(data_more_split).reset_index().drop(['index'], axis=1)

# 3. NER 預測人名
input_id, segment_id, mask_input = encoded(tokenizer, train_ner, maxlen=maxlen_ner)
prediction = ner_model.predict([input_id, segment_id, mask_input])
y_pred = np.argmax(prediction, axis=-1)
people_list = get_name(input_id, y_pred)


# 4. 將拆開的句子組合回去
train_ner['people_list'] = people_list
content = train_ner[['news_id', 'content', 'aml_label']]
content = content.groupby(['news_id', 'aml_label'])['content'].apply(lambda x : '。'.join(x)).reset_index()
people = train_ner[['news_id', 'aml_label', 'people_list']]
people = people.groupby(['news_id', 'aml_label'])['people_list'].agg(sum).reset_index()
people['people_list'] = [list(set(people)) for people in people['people_list']]
train_ner = pd.merge(content, people, on=['news_id', 'aml_label'], how='left')

# 5. 將 [UNK], [PAD] 轉換回來 (王春? -> 王春甡)
for _, row in train_ner.iterrows():
    for i, name in enumerate(row['people_list']):
        if ('?' in name) | ('!' in name):
            reexp = name.replace('?', '.').replace('!', '.')
            reexp = r"{}".format(reexp)
            row['people_list'][i] = re.search(reexp, row['content']).group()
            
AML = predict_sentences_model2(train_ner, list(train_ner['people_list']), tokenizer=tokenizer, maxlen=maxlen_sentences, stick=False)

#???
AML = AML.groupby(['news_id', 'Name'])['Sentence'].apply('，'.join).reset_index()

train_name = train.drop(['content'], axis=1)
s = train_name.apply(lambda x: pd.Series(x['name']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Name'
train_name = train_name.drop('name', axis=1).join(s)

aml_train = pd.merge(AML, train_name, on=['news_id', 'Name'], how='left')
aml_train = aml_train.fillna(0)

In [ ]:
#extra = aml_train[((aml_train.Sentence.str.contains('警')) | (aml_train.Sentence.str.contains('檢察官')))]
#aml_train = aml_train.append(extra).append(extra)

In [ ]:
label = np.asarray(aml_train['aml_label'])

In [ ]:
input_id, segment_id, mask_input = encoded(tokenizer, aml_train, maxlen_sentences)

In [ ]:
#def bert_LSTM_model2():
#    
#    model2 = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, seq_len=maxlen_sentences)
#    sequence_output = model2.layers[-9].output
#    #sequence_output = Lambda(lambda x: x[:, 0])(sequence_output)
#    sequence_output = Bidirectional(LSTM(128, return_sequences=False))(sequence_output)
#    output = Dense(1, activation='sigmoid')(sequence_output)
#    model2 = Model(model2.input, output)
#    
#    for layer in model2.layers:
#        layer.trainable = False
#    model2.layers[-1].trainable = True
#    model2.layers[-2].trainable = True
#    
#    return model2
#
#model2 = bert_LSTM_model2()

In [23]:
def bert_model():
    
    model2 = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, seq_len=maxlen_sentences)
    sequence_output = model2.layers[-6].output
    sequence_output = Dense(64, activation='relu')(sequence_output)
    output = Dense(1, activation='sigmoid')(sequence_output)
    model2 = Model(model2.input, output)
    return model2

model2 = bert_model()

In [ ]:
set_trainable = False
for layer in model2.layers:
    if layer.name == 'Encoder-11-MultiHeadSelfAttention':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
total_steps, warmup_steps = calc_train_steps(
    num_example=train.shape[0],
    batch_size=batch_size,
    epochs=epochs,
    warmup_proportion=0.1,
)

optimizer = AdamWarmup(total_steps, warmup_steps, lr=1e-3, min_lr=1e-5)

In [ ]:
callback_list = [
                 keras.callbacks.EarlyStopping(monitor='val_acc', patience=1)
                 #,ModelCheckpoint(filepath='AML_bert.h5', monitor='val_loss', save_best_only=True)
                ]

In [ ]:
model2.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

model2.fit([input_id, segment_id, mask_input],
          label,
          epochs=epochs,
          batch_size=batch_size,
          #validation_split=0.1,
          #callbacks=callback_list
         )

In [24]:
model2.load_weights(r'C:\Users\rocker\bert model\model2_61.h5')

# 4. 預測

In [25]:
def predict_aml(model, test, aml_threshold):
    
    #第一階段預測，大於aml_threshold者為疑似aml文章
    input_id, segment_id, mask_input = encoded(tokenizer, test, maxlen=maxlen)
    prediction = model.predict([input_id, segment_id, mask_input])
    prediction[prediction >= aml_threshold] = 1
    prediction[prediction < aml_threshold] = 0
    
    return prediction

In [26]:
# 取得名字 (預測結果為onehot的狀態)
def get_name(input_id, y_pred):
    
    label_list = []
    word_dict = {v: k for k, v in token_dict.items()}
    
    for input_data, y in zip(input_id, y_pred):
        people_index = ''.join([str(a) for a in list(y)])
        j = 0
        name_list = []
        split_index = re.findall('[12]2*', people_index)
        name = ''.join([word_dict.get(input_data[index]) for index, value in enumerate(y) if value != 0])
        
        # [UNK], [PAD]會被算成 5 個字元，避免轉換成文字的index因長度不同對不上，故用 1 個字元的其他符號替代
        # 王春甡 -> 王春[UNK] -> 王春?
        name = name.replace('[UNK]','?')
        name = name.replace('[PAD]','!')
        
        for i in split_index:
            name_list.append(name[0+j:len(i)+j])
            j = len(i) + j
            
        name_list = [name for name in name_list]
        label_list.append(list(set(name_list)))
    
    return label_list

In [27]:
def predict_sentences(test, people_list, stick, threshold, tokenizer=tokenizer, maxlen=maxlen_sentences):
    
    time = datetime.now()
    
    AML = pd.DataFrame(columns=['news_id', 'Name', 'Sentence'])
    aml_highrisk = np.asarray(test['content'])
    news_ids = np.asarray(test['news_id'])
    people_list = list(test['people_list'])
    
    for k, (news_id, y_news) in enumerate(zip(news_ids ,aml_highrisk)): 
        #用，。？切分句子
        y_news = y_news.replace('。','=。')
        y_news = y_news.replace('，','*，')
        y_news = y_news.replace('？','+？')
        y_news = y_news.replace('；','{；')
        
        news = re.split('，|。|？|；', y_news)
        
        news = [news.replace('=','。') for news in news]
        news = [news.replace('*','，') for news in news]
        news = [news.replace('+','？') for news in news]
        news = [news.replace('{','；') for news in news]
        news = split2textByName(news, people_list[k])
#############################################################################################################
        for i in range(len(people_list[k])):
            # 找出人名存在的 news index
            index = [index for index, _ in enumerate(news) if people_list[k][i] in _]                                
            x = [people for people in people_list[k] if people_list[k][i] not in people]

            name = [name for name in people_list[k] if name != people_list[k][i]]
            name = [name for name in name if ((len(name)<3) & (name[0] != people_list[k][i][0])) | (len(name)>=3)]
            name.sort(reverse=True)

            #if name != []:
            #    text = [re.sub('|'.join(name), '其他人名', news) for news in news]
            #else:
            #    text = news

            for j in index:

                mid = news[j]

                if name != []:
                    name = [name.replace('?', '\?') for name in name]
                    mid = re.sub('|'.join(name), '', mid)
                
                if j == 0:
                    if len(news) != 1:
                        end = news[j+1]  

                        if True in [people in news[j+1] for people in x]:
                            sentences = mid
                        else:
                            sentences = mid + end
                    elif len(news) == 1:
                        sentences = mid

                elif j+1 == len(news):
                    start = news[j-1]

                    if True in [people in news[j-1] for people in x]:
                        sentences = mid
                    else:
                        sentences = start + mid

                else:
                    end = news[j+1]
                    start = news[j-1]                    

                    if '。' in start:
                        start = ''
                    elif '。' in mid:
                        end = ''                 

                    if (True not in [people in news[j-1] for people in x]) & (True not in [people in news[j+1] for people in x]):
                        sentences = start + mid + end
                    elif (True not in [people in news[j-1] for people in x]) & (True in [people in news[j+1] for people in x]):
                        sentences = start + mid
                    elif (True in [people in news[j-1] for people in x]) & (True not in [people in news[j+1] for people in x]):
                        sentences = mid + end
                    else:
                        sentences = mid    
   
                AML = AML.append(pd.DataFrame([[news_ids[k] ,people_list[k][i], sentences]], columns=AML.columns))
                #break  
              
                
    print('1.提取句子', datetime.now() - time)
    time = datetime.now()
    
    #若 stick==True 則把多筆同姓名句子以逗點合併 (效果不好)
    if stick:
        AML = AML.groupby(['news_id', 'Name'])['Sentence'].apply('，'.join).reset_index()

    
    # 把指向同一人的姓名改成一樣（陳男 -> 陳水扁），若指向多人則不改（陳男 -> 陳致中、陳水扁）
    # 將預測不完整的名字回填（王音 -> 王音之）
    name_list = []
    for ids in AML['news_id'].unique():
        full_name = [name for name in AML[(AML['news_id'] == ids)]['Name']]
        full_3name = [name for name in AML[(AML['news_id'] == ids)]['Name'] if len(name) == 3]
        
        a = Counter([name[0] for name in full_3name])
        keep = [k for k,v in a.items() if v == 1]
        full_3name_filter = [name for name in full_3name if name[0] in keep]
        name_dict = dict((name[0], name) for name in full_3name_filter)   # ex: {'陳' : '陳水扁'}

        name_dict_2 = dict(zip([name[0:2] for name in full_3name], full_3name))  # ex: {'王音': '王音之'}
        name_dict_3 = dict(zip([name[1:] for name in full_3name], full_3name))
        for name in full_name:
            if (name[0] in name_dict.keys()) & (len(name) == 1):
                name_list.append(name_dict.get(name[0]))
            elif (name[0] in name_dict.keys()) & (len(name) == 2) & (name[-1] in ['男', '嫌', '婦', '夫', '某', '女', '妻',\
                                                                                  '員', '稱', '家', '哥', '媽', '生', '處',\
                                                                                  '和', '揆', '要', '再', '董', '涉', '母',\
                                                                                  '辱', '公', '少', '為', '指', '翁', '粉',\
                                                                                  '趁', '仔', '依', '氏', '父']):
                name_list.append(name_dict.get(name[0]))
            elif (name in name_dict_2.keys()) & (len(name) == 2):
                name_list.append(name_dict_2.get(name))
            elif (name in name_dict_3.keys()):
                name_list.append(name_dict_3.get(name))
            else:
                name_list.append(name)
                
    print('2.整理名字', datetime.now() - time)
    time = datetime.now()
    
    # 排除重複資料、排除一字、兩字簡稱、兩字三字四字姓不在姓氏表中的人
    AML['Name'] = name_list
    AML = AML.drop_duplicates()
    AML = AML[AML['Name'].apply(lambda x: (len(x) > 1) )]
    AML = AML[~AML['Name'].apply(lambda x: (len(x) == 2) & (x[1] in ['男', '嫌', '婦', '夫', '某', '女', '妻',\
                                                                     '員', '稱', '家', '哥', '媽', '生', '處',\
                                                                     '和', '揆', '要', '再', '董', '涉', '母',\
                                                                     '辱', '公', '少', '為', '指', '翁', '粉',\
                                                                     '趁', '仔', '依', '氏', '父']))]
    
    
    print('3.刪除名字', datetime.now() - time)
    time = datetime.now()
    
    # 第二階段 預測句子
    input_id, segment_id, mask_input = encoded(tokenizer, AML, maxlen=maxlen)
    prediction = model2.predict([input_id, segment_id, mask_input])
    AML['prediction'] = prediction
    
    
    # 同一人只要有一筆資料大於閥值（max），則預測為 aml 人物；若新聞中無人大於閥值，則為非 aml 新聞
    AML['prediction'] = AML['prediction'].apply(lambda x: 0 if x < threshold else 1)
    AML = AML.groupby(['news_id', 'Name'])['prediction'].max().reset_index()
    AML = AML[AML['prediction'] == 1]
    AML = AML.groupby(['news_id','prediction'])['Name'].apply(list).reset_index()
    
    print('4.預測名字', datetime.now() - time)
    
    return AML

In [28]:
def f1_score(a, b):
    
    if (len(a) != 0) & (len(b) != 0):
        recall = float(len(set(a) & set(b)) / len(a))
        pecision = float(len(set(a) & set(b)) / len(b))
        score = 2 / (np.reciprocal(recall) + np.reciprocal(pecision))
        return score
    elif (len(a) == 0) & (len(b) == 0):
        return 1
    else:
        return 0

# 5. 網格搜索

### [NER法]

In [29]:
# 把大於512的新聞以句點分段 (bert最多只能吃512)
def split_content(data):
    data_more_split = pd.DataFrame()
    for i, row in data.iterrows():
        if (len(row['content']) > 512) & (len(row['content']) <= 1024):

            s = row['content']
            s_split = [(i, abs(len(s)//2 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left = min(s_split, key=lambda x: x[1])[0]
            first = "。".join([s_split[i][2] for i in range(idx_left)])
            second = "。".join([s_split[i][2] for i in range(idx_left, len(s_split))])    
            contents = [first, second]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content, 'aml_label':row['aml_label'], 'prediction':row['prediction']}, index=[66]), ignore_index=True)

        elif len(row['content']) > 1024:

            s = row['content']
            s_split1 = [(i, abs(len(s)//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            s_split2 = [(i, abs(len(s)*2//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left1 = min(s_split1, key=lambda x: x[1])[0]
            idx_left2 = min(s_split2, key=lambda x: x[1])[0]
            first = "。".join([s_split1[i][2] for i in range(idx_left1)])
            second = "。".join([s_split1[i][2] for i in range(idx_left1, idx_left2)])
            third = "。".join([s_split1[i][2] for i in range(idx_left2, len(s_split1))])
            contents = [first, second, third]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content, 'aml_label':row['aml_label'], 'prediction':row['prediction']}, index=[66]), ignore_index=True)
    
    return data_more_split

In [30]:
def GridSearch_ner(df, model, test, aml_threshold, stick, threshold):
    
    time = datetime.now()
    
    # 1. 預測是否疑似 aml
    prediction = predict_aml(model, test=test, aml_threshold=aml_threshold)
    test['prediction'] = prediction
    aml_highrisk = test[test['prediction'] == 1]
    
    # 2. 將超過 512 的句子以句點拆成多句分段預測
    test_ner = aml_highrisk.drop(['name'], axis=1)
    data_less = test_ner[test_ner['content'].str.len() <= 512]
    data_more = test_ner[test_ner['content'].str.len() > 512]
    data_more_split = split_content(data_more)
    test_ner = data_less.append(data_more_split).reset_index().drop(['index'], axis=1)
    
    # 3. NER 預測人名
    input_id, segment_id, mask_input = encoded(tokenizer, test_ner, maxlen=maxlen_ner)
    prediction = ner_model.predict([input_id, segment_id, mask_input])
    y_pred = np.argmax(prediction, axis=-1)
    people_list = get_name(input_id, y_pred)
    
    
    # 4. 將拆開的句子組合回去
    test_ner['people_list'] = people_list
    content = test_ner[['news_id', 'content', 'aml_label', 'prediction']]
    content = content.groupby(['news_id', 'aml_label', 'prediction'])['content'].apply(lambda x : '。'.join(x)).reset_index()
    people = test_ner[['news_id', 'aml_label', 'prediction', 'people_list']]
    people = people.groupby(['news_id', 'aml_label', 'prediction'])['people_list'].agg(sum).reset_index()
    people['people_list'] = [list(set(people)) for people in people['people_list']]
    test_ner = pd.merge(content, people, on=['news_id', 'aml_label', 'prediction'], how='left')
    
    # 5. 將 [UNK], [PAD] 轉換回來 (王春? -> 王春甡)
    for _, row in test_ner.iterrows():
        for i, name in enumerate(row['people_list']):
            if ('?' in name) | ('!' in name):
                reexp = name.replace('?', '.').replace('!', '.')
                reexp = re.compile(reexp,re.IGNORECASE)
                row['people_list'][i] = re.search(reexp, row['content']).group()
    
    
    print('0.CKIP', datetime.now() - time)
    
    # 6. 判斷名字前後句使是否為 aml
    AML = predict_sentences(test_ner, list(test_ner['people_list']), tokenizer=tokenizer, maxlen=maxlen_sentences, stick=stick, threshold=threshold)
    
    test_prediction = pd.merge(test, AML[['news_id', 'Name']], on='news_id', how='left')
    test_prediction['Name'] = test_prediction['Name'].apply(lambda x: x if isinstance(x, list) else [])
    test_prediction['text_prediction'] = test_prediction['Name'].apply(lambda x: 0 if x == [] else 1)
    test_prediction = test_prediction.drop(['content'],axis = 1)
    test_prediction.columns = ['news_id', 'name', 'label', 'AML_prediction', 'Name_prediction', 'text_prediction']
    
    # 7. 算分數
    score = []
    for i in range(len(test_prediction)):
        temp = f1_score(test_prediction['name'][i], test_prediction['Name_prediction'][i])
        score.append(temp)
        
    test_prediction['f1_score'] = score    
    total_score = sum(score)
    aml_score = sum(test_prediction[test_prediction['label'] == 1]['f1_score'])    
    
    df = df.append(pd.DataFrame([[aml_threshold, threshold, stick, total_score, aml_score]], columns=df.columns))
    
    print('aml_threshold =', aml_threshold, 'stick =', stick, 'threshold =', threshold, 'total_score =', total_score, 'aml_score =', aml_score)

    return df

In [31]:
df = pd.DataFrame(columns=['aml_threshold', 'threshold', 'stick', 'total_score', 'aml_score'])

In [32]:
for aml_threshold in [0.4]:
    for threshold in [0.4]:
        for stick in [False]:        
            df = GridSearch_ner(df, model, test, aml_threshold=aml_threshold, stick=stick, threshold=threshold)

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


0.CKIP 0:00:08.457608
1.提取句子 0:00:01.453028
2.整理名字 0:00:00.048930
3.刪除名字 0:00:00.003963
4.預測名字 0:00:08.713954
aml_threshold = 0.4 stick = False threshold = 0.4 total_score = 80.34603174603177 aml_score = 61.346031746031734


In [33]:
len(test)

87

In [ ]:
content = '收受、樂揚建設總經理鄒雪娥贈送的2支香奈兒名表、蕭邦名表；還在的生日過後幾個月，'
input_id, segment_id = tokenizer.encode(content, max_len=maxlen)
mask_input = [transfer(i) for i in input_id]
prediction = model.predict([[input_id], [segment_id], [mask_input]])
prediction

In [ ]:
content = '收受、樂揚建設總經理鄒雪娥贈送的2支香奈兒名表、蕭邦名表；還在的生日過後幾個月，'
input_id, segment_id = tokenizer.encode(content, max_len=maxlen)
mask_input = [transfer(i) for i in input_id]
prediction = model2.predict([[input_id], [segment_id], [mask_input]])
prediction

In [ ]:
data2 = pd.read_csv(r'C:\Users\rocker\extra data\人肉爬蟲_20200615.csv', encoding='cp950')
data2['name'] = data2['name'].apply(lambda x: eval(x))
data2['content'] = data2['content'].apply(lambda x: re.sub('<[^>]*>|【[^】]*】|（[^）]*）|〔[^〕]*〕', '', x))
data2['content'] = data2['content'].apply(lambda x: x.strip().replace('記者', '＜')
                                                   .replace('報導', '＞')
                                                   .replace('▲', '')\
                                                   .replace('。　', '。')\
                                                   .replace('', '')\
                                                   .replace('.', '')\
                                                   .replace(' ', '')\
                                                   .replace('“', '「')\
                                                   .replace('”', '」'))
data2['content'] = data2['content'].apply(lambda x: re.sub('＜[^＞]*＞', '', x))
test = data2
test = test[['news_id', 'name', 'content', 'aml_label']]

In [ ]:
data2 = pd.read_csv(r'E:\Desktop\檢察官_法官_filter3 - 複製.csv', encoding='cp950')
data2['aml_label'] = data2['people_list'].apply(lambda x: 0 if x == '[]' else 1)
data2 = data2.drop(['title', 'link'], axis=1)
data2['people_list'] = data2['people_list'].apply(lambda x: eval(x))
data2['news_id'] = range(10000, 10000+len(data2))
data2['content'] = data2['content'].apply(lambda x: re.sub('<[^>]*>|【[^】]*】|（[^）]*）|〔[^〕]*〕', '', x))
data2['content'] = data2['content'].apply(lambda x: x.strip().replace('記者', '＜')
                                                   .replace('報導', '＞')
                                                   .replace('▲', '')\
                                                   .replace('。　', '。')\
                                                   .replace('', '')\
                                                   .replace('.', '')\
                                                   .replace(' ', '')\
                                                   .replace('“', '「')\
                                                   .replace('”', '」'))
data2['content'] = data2['content'].apply(lambda x: re.sub('＜[^＞]*＞', '', x))
data2.columns = ['news_id', 'content', 'aml_label', 'name']

In [ ]:
data3 = pd.read_csv(r'E:\Desktop\檢察官_法官_filter.csv', encoding='cp950')
data3 = data3.drop(['title', 'link'], axis=1)
data3['aml_label'] = data3['people_list'].apply(lambda x: 0 if x == '[]' else 1)
data3['people_list'] = data3['people_list'].apply(lambda x: eval(x))
data3['news_id'] = range(10000, 10000+len(data3))
data3.columns = ['news_id', 'content', 'aml_label', 'name']


In [ ]:
test = test.append(data2).append(data3)

In [ ]:
test.news_id = range(0, len(test))